In [1]:
import os
import imageio
import tensorflow as tf
import glob

def tfevents_image_to_gif(log_dir, gif_dir = 'gifs/', mode = 'testing', tag = 'logging/confusion_matrix/image/0', frame_duration = 0.1):

    boldstr = lambda s: '\x1b[1;03m' + s + '\x1b[0m'
    
    if not os.path.exists(gif_dir):
        os.makedirs(gif_dir)
    
    fn = glob.glob(log_dir  + mode + '/*')[0]

    str_flag = 'tfevents.'
    flag_idx = fn.find(str_flag)
    temp_fn = fn[flag_idx + len(str_flag):]
    fn_end = temp_fn.find('.')
    gif_name = gif_dir + log_dir[:-1] + '_' + mode + '_' + temp_fn[:fn_end] + '.gif'
    
    if not os.path.exists(gif_name):
#         os.makedirs(gif_dir)
        image_str = tf.placeholder(tf.string)
        im_tf = tf.image.decode_image(image_str)
        images = []
        sess = tf.Session()
        with sess.as_default():
            count = 0
            for e in tf.train.summary_iterator(fn):
                for v in e.summary.value:
                    if v.tag == tag:
                        im = im_tf.eval({image_str: v.image.encoded_image_string})
                        images.append(im)
                        count += 1  
        imageio.mimsave(gif_name, images, duration=frame_duration, subrectangles = True) 
        print(gif_name + ' saved')


    else:
        print(gif_name + boldstr(' exists'))

    
    return None

def gif_all_logs():
    log_dirs = [s for s in glob.glob('*/') if s not in ['gifs/']]
    for log_dir in log_dirs:
        tfevents_image_to_gif(log_dir, mode = 'testing')
        tfevents_image_to_gif(log_dir, mode = 'training')

In [5]:
tfevents_image_to_gif('positive_negative_classifier_cm/', mode = 'testing')

gifs/positive_negative_classifier_cm_testing_1533151064.gif saved


In [3]:
#gif_all_logs()